<a href="https://colab.research.google.com/github/PabloMendieta03/UFV-VisualizacionDeDatos/blob/main/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Definición del mensaje que se quier transmitir.
* Conocer los datos que apoyan el gr

In [7]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Especicicamos la ruta del archivo CSV en Google Drive
ruta_archivo_drive = '/content/drive/MyDrive/datos_ejercicio_ventas.csv'

# Cargamos el archivo CSV en un DataFrame de pandas
data = pd.read_csv(ruta_archivo_drive)

# mostrar los datos
display(data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,7.543562e+05
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,5.600306e+05
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,8.850198e+04
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,3.632245e+05
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,3.961761e+05
...,...,...,...,...,...,...,...,...
18661,Great Britain,Pepsi Regular (L3),2024,2,AI_forecast,AI_P10F,2023.0,1.313511e+06
18662,Hungary,Pepsi Regular (L3),2024,7,AI_forecast,AI_P07F,2023.0,1.314395e+06
18663,Norway,7up (L3),2024,1,AI_forecast,AI_P05F,2023.0,0.000000e+00
18664,Portugal,Lipton (L3),2024,3,AI_forecast,AI_P02F,2023.0,5.330634e+05
